# SQLite Tutorial Python Standard Library

In this tutorial, you will create a database of Monty Python movies using basic sqlite3 functionality. It assumes a fundamental understanding of database concepts, including [cursors](https://en.wikipedia.org/wiki/Cursor_(databases)) and [transactions](https://en.wikipedia.org/wiki/Database_transaction).

First, we need to create a new database and open a database connection to allow sqlite3 to work with it. Call [`sqlite3.connect()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.connect) to create a connection to the database `movies.db` in the current working directory, implicitly creating it if it does not exist:

In [2]:
import sqlite3
from pathlib import Path

In [13]:
db_path = Path('../data') / 'monty_python_movies.db'
connection = sqlite3.connect(db_path)

The returned Connection object con represents the connection to the on-disk database.

In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor. Call con.cursor() to create the Cursor:

In [14]:
cursor = connection.cursor()

Now that we’ve got a database connection and a cursor, we can create a database table movie with columns for title, release year, and review score. For simplicity, we can just use column names in the table declaration – thanks to the [flexible typing](https://www.sqlite.org/flextypegood.html) feature of SQLite, specifying the data types is optional. Execute the CREATE TABLE statement by calling [cur.execute(...)](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute):

In [15]:
cursor.execute("CREATE TABLE movie(title, year, score)")

We can verify that the new table has been created by querying the sqlite_master table built-in to SQLite, which should now contain an entry for the movie table definition (see [The Schema Table](https://www.sqlite.org/schematab.html) for details). Execute that query by calling [cur.execute(...)](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute), assign the result to res, and call [res.fetchone()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchone) to fetch the resulting row:

In [16]:
result = cursor.execute("SELECT name FROM sqlite_master")
print(result.fetchone())

('movie',)


We can see that the table has been created, as the query returns a tuple containing the table’s name. If we query sqlite_master for a non-existent table spam, [res.fetchone()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchone) will return None:

In [17]:
result = cursor.execute("SELECT name FROM sqlite_master WHERE name = 'spam'")
result.fetchone() is None

True

Now, add two rows of data supplied as SQL literals by executing an INSERT statement, once again by calling [cur.execute(...)](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute):

In [18]:
cursor.execute("""
   INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5) 
""")

The INSERT statement implicitly opens a transaction, which needs to be committed before changes are saved in the database (see [Transaction control](https://docs.python.org/3/library/sqlite3.html#sqlite3-controlling-transactions) for details). Call [con.commit()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.commit) on the connection object to commit the transaction:

In [19]:
connection.commit()

We can verify that the data was inserted correctly by executing a SELECT query. Use the now-familiar [cur.execute(...)](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute) to assign the result to res, and call [res.fetchall()](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.fetchall) to return all resulting rows:

In [20]:
result = cursor.execute("SELECT score FROM movie")
result.fetchall()

[(8.2,), (7.5,)]

The result is a list of two tuples, one per row, each containing that row’s score value.

Now, insert three more rows by calling [cur.executemany(...)](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany):

In [21]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cursor.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
connection.commit() # Remember to commit the transaction after executing INSERT.

Notice that ? placeholders are used to bind data to the query. Always use placeholders instead of [string formatting](https://docs.python.org/3/tutorial/inputoutput.html#tut-formatting) to bind Python values to SQL statements, to avoid [SQL injection attacks](https://en.wikipedia.org/wiki/SQL_injection) (see [How to use placeholders to bind values in SQL queries](https://docs.python.org/3/library/sqlite3.html#sqlite3-placeholders) for more details).

We can verify that the new rows were inserted by executing a SELECT query, this time iterating over the results of the query:

In [22]:
for row in cursor.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


In [23]:
connection.close()
new_connection = sqlite3.connect(db_path)
new_cursor = new_connection.cursor()
result = new_cursor.execute("SELECT title, year FROM movie ORDER BY score DESC")
title, year = result.fetchone()
print(f"The highest scoring Monty Python movie is {title!r}, released in {year}")

The highest scoring Monty Python movie is 'Monty Python and the Holy Grail', released in 1975


In [24]:
new_connection.close()

You’ve now created an SQLite database using the sqlite3 module, inserted data and retrieved values from it in multiple ways.